In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from pprint import pprint

In [2]:
# Visit each url
url = "https://redplanetscience.com/"
url1 = "https://spaceimages-mars.com/"
url2 = "https://galaxyfacts-mars.com/"
url3 = 'https://marshemispheres.com/'

In [3]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\Rodrigo\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


In [4]:
# Scrape page news into Soup
browser.visit(url)
time.sleep(1)
html = browser.html
soup = bs(html, "html.parser")

div = soup.find('div', {'class':'list_text'})
title = div.find('div',{'class':'content_title'})
article = div.find('div',{'class':'article_teaser_body'})

news_title = title.text
news_p = article.text

In [5]:
print(news_title)
print(news_p)

NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light
Vast areas of the Martian night sky pulse in ultraviolet light, according to images from NASA’s MAVEN spacecraft. The results are being used to illuminate complex circulation patterns in the Martian atmosphere.


In [6]:
# Scrape feature image
browser.visit(url1)
time.sleep(1)
html = browser.html
soup = bs(html, "html.parser")

img = soup.find('img',{'class':'headerimage fade-in'})
src = img['src']
featured_image_url = url1+src

In [7]:
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


In [8]:
# Scrape facts into pandas
table = pd.read_html(url2)

# Select mars facts table
mars_df = table[0]
mars_df = mars_df.rename(columns={0: 'Description', 1:'Mars', 2:'Earth'})

mars_df

,Description,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [9]:
# Convert to html
mars_table = mars_df.to_html(index=False)
mars_table_clean = mars_table.replace('\n', '')

pprint(mars_table_clean)

('<table border="1" class="dataframe">  <thead>    <tr style="text-align: '
 'right;">      <th>Description</th>      <th>Mars</th>      <th>Earth</th>    '
 '</tr>  </thead>  <tbody>    <tr>      <td>Mars - Earth Comparison</td>      '
 '<td>Mars</td>      <td>Earth</td>    </tr>    <tr>      '
 '<td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    '
 '</tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      '
 '<td>5.97 × 10^24 kg</td>    </tr>    <tr>      <td>Moons:</td>      '
 '<td>2</td>      <td>1</td>    </tr>    <tr>      <td>Distance from '
 'Sun:</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    '
 '</tr>    <tr>      <td>Length of Year:</td>      <td>687 Earth '
 'days</td>      <td>365.24 days</td>    </tr>    <tr>      '
 '<td>Temperature:</td>      <td>-87 to -5 °C</td>      <td>-88 to '
 '58°C</td>    </tr>  </tbody></table>')


In [10]:
# Scrape Hemispheres Images
browser.visit(url3)
time.sleep(1)
html = browser.html
soup = bs(html, "html.parser")

In [11]:
# Get images
hemispheres = soup.find_all('div',{'class':'item'})
hemisphere_image_url = []

In [12]:
for hemisphere in hemispheres:
    hemisphere_dict = {}
    
    # Get title
    hemisphere_title = hemisphere.find('h3').text
    
    # Clicking link for each title to get image
    browser.links.find_by_partial_text(hemisphere_title).click()
    html = browser.html
    soup = bs(html, "html.parser")
    
    # Scrape image link
    hemis_div = soup.find('div', {'class':'wide-image-wrapper'})
    hemis_img = hemis_div.find('img',{'class':'wide-image'})['src']
    hemis_url = url3 + hemis_img
    
    # Fill dictionary
    hemisphere_dict['title'] = hemisphere_title
    hemisphere_dict['url'] = hemis_url
    
    # Append to list
    hemisphere_image_url.append(hemisphere_dict)
    
    #Set up browser to go back for each image
    browser.back()

In [13]:
# Check hesmisphere_images_url
print(hemisphere_image_url)

[{'title': 'Cerberus Hemisphere Enhanced', 'url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
